In [1]:
import os
from bs4 import BeautifulSoup as bs
import requests
from splinter import Browser
import pandas as pd
import pprint
import datetime as dt

In [2]:
url_corona = "https://www.worldometers.info/coronavirus/#countries"
html_corona = requests.get(url_corona)
html_corona = html_corona.text
#html_corona

In [3]:
soup_corona = bs(html_corona, 'html.parser')

In [4]:
corona_table = soup_corona.find(class_="table-bordered")

corona_table = str(corona_table)


In [5]:
corona_df = pd.read_html(corona_table)[0]

#replacing blanks with 0
corona_df.fillna(0,inplace = True)

#removing the total row)
corona_df = corona_df[~corona_df['Country,Other'].isin(['Total:', 'Total', 'total'])]
corona_df.to_csv('new_data.csv')
#replace items using wild card and regex to have that applied
corona_df["NewCases"].replace("\,","",regex = True, inplace = True)
corona_df["NewCases"].replace("\+","", regex = True, inplace = True)
corona_df["NewCases"] = corona_df["NewCases"].astype("float")
corona_df["NewDeaths"].replace("\,","",regex = True, inplace = True)
corona_df["NewDeaths"].replace("\+","", regex = True, inplace = True)
corona_df["NewDeaths"] = corona_df["NewDeaths"].astype("float")


#add date column for date data scrped
corona_df["Date"]= dt.datetime.today()

In [6]:
corona_df

,"Country,Other",TotalCases,NewCases,TotalDeaths,NewDeaths,TotalRecovered,ActiveCases,"Serious,Critical",Tot Cases/1M pop,Deaths/1M pop,TotalTests,Tests/ 1M pop,Date
0,World,2966489,47085.0,205928.0,2764.0,873857.0,1886704,57553.0,381.00,26.4,0.0,0.0,2020-04-26 13:22:21.585693
1,USA,970757,10106.0,54941.0,685.0,118633.0,797183,15116.0,2933.00,166.0,5353358.0,16173.0,2020-04-26 13:22:21.585693
2,Spain,226629,2870.0,23190.0,288.0,117727.0,85712,7764.0,4847.00,496.0,930230.0,19896.0,2020-04-26 13:22:21.585693
3,Italy,197675,2324.0,26644.0,260.0,64928.0,106103,2009.0,3269.00,441.0,1757659.0,29071.0,2020-04-26 13:22:21.585693
4,France,161488,0.0,22856.0,242.0,44903.0,93729,4682.0,2474.00,350.0,463662.0,7103.0,2020-04-26 13:22:21.585693
...,...,...,...,...,...,...,...,...,...,...,...,...,...
208,Sao Tome and Principe,4,0.0,0.0,0.0,0.0,4,0.0,18.00,0.0,19.0,87.0,2020-04-26 13:22:21.585693
209,Anguilla,3,0.0,0.0,0.0,1.0,2,0.0,200.00,0.0,0.0,0.0,2020-04-26 13:22:21.585693
210,Saint Pierre Miquelon,1,0.0,0.0,0.0,0.0,1,0.0,173.00,0.0,0.0,0.0,2020-04-26 13:22:21.585693
211,Yemen,1,0.0,0.0,0.0,1.0,0,0.0,0.03,0.0,120.0,4.0,2020-04-26 13:22:21.585693


In [7]:
# adding lat long info 
countries_df = pd.read_csv("country_lan_lng.csv",engine='python')
merged = corona_df.merge(countries_df, left_on='Country,Other', right_on='name')
df_used = merged.drop(countries_df.columns[0:-2], axis=1)

df_used.rename(columns={"Country,Other":"Country","Serious,Critical": "CriticalCases"}, inplace = True)
#replace items using wild card and regex to have that applied
df_used["NewCases"].replace("\,","",regex = True, inplace = True)
df_used["NewCases"].replace("\+","", regex = True, inplace = True)
df_used["ClosedCases"] = df_used["TotalDeaths"] +  df_used["TotalRecovered"]

df_used["ClosedDeathRate"] = df_used["TotalDeaths"]/df_used["ClosedCases"]
df_used["ClosedDeathRate"].fillna(0,inplace = True)

df_used["CurrentDeathRate"] = df_used["TotalDeaths"]/df_used["TotalCases"]
df_used["CurrentDeathRate"].fillna(0,inplace = True)

total_cases = df_used["TotalCases"].sum()
total_closed_cases = df_used["ClosedCases"].sum()

df_used["Weight"] = [x/total_closed_cases for x in df_used["ClosedCases"]]

# #checking if total weight is 1 
# validate_weight = df_used["Weight"].sum()


df_used["DeathRateXWeight"] = df_used["CurrentDeathRate"] *df_used["Weight"]
df_used["DeathRateXWeight"].sum()
df_used["notes"] = ["Last 8 columns are added by Roaya" for x in df_used["DeathRateXWeight"]  ]
df_used.sort_values(by = "DeathRateXWeight", ascending = False)

,Country,TotalCases,NewCases,TotalDeaths,NewDeaths,TotalRecovered,ActiveCases,CriticalCases,Tot Cases/1M pop,Deaths/1M pop,...,Tests/ 1M pop,Date,Longitude,Latitude,ClosedCases,ClosedDeathRate,CurrentDeathRate,Weight,DeathRateXWeight,notes
1,Spain,226629,2870.0,23190.0,288.0,117727.0,85712,7764.0,4847.0,496.0,...,19896.0,2020-04-26 13:22:21.585693,-3.647550,40.244487,140917.0,0.164565,0.102326,0.130914,0.013396,Last 8 columns are added by Roaya
2,Italy,197675,2324.0,26644.0,260.0,64928.0,106103,2009.0,3269.0,441.0,...,29071.0,2020-04-26 13:22:21.585693,12.070013,42.796626,91572.0,0.290962,0.134787,0.085072,0.011467,Last 8 columns are added by Roaya
0,USA,970757,10106.0,54941.0,685.0,118633.0,797183,15116.0,2933.0,166.0,...,16173.0,2020-04-26 13:22:21.585693,-97.922211,39.381266,173574.0,0.316528,0.056596,0.161253,0.009126,Last 8 columns are added by Roaya
3,France,161488,0.0,22856.0,242.0,44903.0,93729,4682.0,2474.0,350.0,...,7103.0,2020-04-26 13:22:21.585693,2.618787,47.824905,67759.0,0.337313,0.141534,0.062949,0.008909,Last 8 columns are added by Roaya
7,Iran,90481,1153.0,5710.0,60.0,69657.0,15114,3079.0,1077.0,68.0,...,5016.0,2020-04-26 13:22:21.585693,54.274070,32.575033,75367.0,0.075763,0.063107,0.070017,0.004419,Last 8 columns are added by Roaya
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
139,Mozambique,76,6.0,0.0,0.0,12.0,64,0.0,2.0,0.0,...,53.0,2020-04-26 13:22:21.585693,35.533675,-17.273816,12.0,0.000000,0.000000,0.000011,0.000000,Last 8 columns are added by Roaya
116,Vietnam,270,0.0,0.0,0.0,225.0,45,8.0,3.0,0.0,...,2188.0,2020-04-26 13:22:21.585693,106.299147,16.646017,225.0,0.000000,0.000000,0.000209,0.000000,Last 8 columns are added by Roaya
127,Cambodia,122,0.0,0.0,0.0,117.0,5,1.0,7.0,0.0,...,345.0,2020-04-26 13:22:21.585693,104.906943,12.720048,117.0,0.000000,0.000000,0.000109,0.000000,Last 8 columns are added by Roaya
125,Madagascar,124,1.0,0.0,0.0,71.0,53,1.0,4.0,0.0,...,85.0,2020-04-26 13:22:21.585693,46.704737,-19.371896,71.0,0.000000,0.000000,0.000066,0.000000,Last 8 columns are added by Roaya


In [8]:
df_used.to_csv('data.csv')
df_used.to_html('data.html')

In [9]:
corona_json = df_used.to_json(orient='records')
df_used.to_json('data.json',orient='records')

In [10]:
corona_json

'[{"Country":"USA","TotalCases":970757,"NewCases":10106.0,"TotalDeaths":54941.0,"NewDeaths":685.0,"TotalRecovered":118633.0,"ActiveCases":797183,"CriticalCases":15116.0,"Tot\\u00a0Cases\\/1M pop":2933.0,"Deaths\\/1M pop":166.0,"TotalTests":5353358.0,"Tests\\/ 1M pop":16173.0,"Date":1587907341585,"Longitude":-97.922211,"Latitude":39.381266,"ClosedCases":173574.0,"ClosedDeathRate":0.316527821,"CurrentDeathRate":0.0565960379,"Weight":0.1612525327,"DeathRateXWeight":0.0091262545,"notes":"Last 8 columns are added by Roaya"},{"Country":"Spain","TotalCases":226629,"NewCases":2870.0,"TotalDeaths":23190.0,"NewDeaths":288.0,"TotalRecovered":117727.0,"ActiveCases":85712,"CriticalCases":7764.0,"Tot\\u00a0Cases\\/1M pop":4847.0,"Deaths\\/1M pop":496.0,"TotalTests":930230.0,"Tests\\/ 1M pop":19896.0,"Date":1587907341585,"Longitude":-3.64755047,"Latitude":40.24448698,"ClosedCases":140917.0,"ClosedDeathRate":0.1645649567,"CurrentDeathRate":0.1023258277,"Weight":0.1309137495,"DeathRateXWeight":0.013395